## Let's mount google drive first for a storage and GPU

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# make sure you use Tesla K80 GPU (12GB), by changing 'runtmie type' 
!nvidia-smi

Fri Mar 10 19:43:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Clone Darknet GitHub repository

In [3]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'
!rm -fr darknet
!git clone https://github.com/AlexeyAB/darknet/

Cloning into 'darknet'...
remote: Enumerating objects: 15514, done.
remote: Total 15514 (delta 0), reused 0 (delta 0), pack-reused 15514
Receiving objects: 100% (15514/15514), 14.16 MiB | 15.35 MiB/s, done.
Resolving deltas: 100% (10413/10413), done.


In [4]:
!apt install gcc-5 g++-5 -y
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc 
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package g++-5 is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  gcc-10-test-results gcc-9-test-results gcc-8-test-results gcc-7-test-results

Package gcc-5 is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'gcc-5' has no installation candidate
E: Package 'g++-5' has no installation candidate


## Go to the darknet folder, enable GPU, then compile the entire thing

In [5]:
%cd darknet
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1147 |             if (iteration_old == 0)
      |             ^~
./src/imag

## Get the YOLO Weights file

In [6]:
!wget https://pjreddie.com/media/files/yolov3.weights
!chmod a+x ./darknet

--2023-03-10 19:46:45--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  17.2MB/s    in 16s     

2023-03-10 19:47:02 (14.9 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



## To get the working Directory

In [7]:
!pwd

/content/darknet


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!apt install ffmpeg libopencv-dev libgtk-3-dev python-numpy python3-numpy libdc1394-22 libdc1394-22-dev libjpeg-dev libtiff5-dev libavcodec-dev libavformat-dev libswscale-dev libxine2-dev libgstreamer1.0-dev libgstreamer-plugins-base1.0-dev libv4l-dev libtbb-dev qtbase5-dev libfaac-dev libmp3lame-dev libopencore-amrnb-dev libopencore-amrwb-dev libtheora-dev libvorbis-dev libxvidcore-dev x264 v4l-utils unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libjpeg-dev is already the newest version (8c-2ubuntu8).
libjpeg-dev set to manually installed.
libvorbis-dev is already the newest version (1.3.6-2ubuntu1).
libvorbis-dev set to manually installed.
libdc1394-22 is already the newest version (2.2.5-2.1).
libdc1394-22 set to manually installed.
libdc1394-22-dev is already the newest version (2.2.5-2.1).
libdc1394-22-dev set to manually installed.
libtbb-dev is already the newest version (2020.1-2).
libtbb-dev set to manually installed.
unzip is already the newest version (6.0-25ubuntu1.1).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
libavcodec-dev is already the newest version (7:4.2.7-0ubuntu0.1).
libavcodec-dev set to manually installed.
libavformat-dev is already the newest version (7:4.2.7-0ubuntu0.1).
libavformat-dev set to manually installed.
libswscale-dev is already the newest version (7:4.2.7-0ubuntu0.1).
libswscale-de

## It's time to upload the video

In [13]:
from google.colab import files

uploaded = files.upload()

Saving harsha_intro.mp4 to harsha_intro.mp4


In [14]:
!./darknet detector demo cfg/coco.data cfg/yolov3.cfg yolov3.weights -dont_show harsha_intro.mp4 -i 0 -out_filename output1.avi -thresh 0.7

Streaming output truncated to the last 5000 lines.

person: 100% 

FPS:36.0 	 AVG_FPS:35.2

 cvWriteFrame 
Objects:

person: 100% 

FPS:35.9 	 AVG_FPS:35.2

 cvWriteFrame 
Objects:

person: 100% 

FPS:35.6 	 AVG_FPS:35.2

 cvWriteFrame 
Objects:

person: 100% 

FPS:35.6 	 AVG_FPS:35.2

 cvWriteFrame 
Objects:

person: 100% 

FPS:35.5 	 AVG_FPS:35.2

 cvWriteFrame 
Objects:

person: 100% 

FPS:35.5 	 AVG_FPS:35.2

 cvWriteFrame 
Objects:

person: 100% 

FPS:35.5 	 AVG_FPS:35.2

 cvWriteFrame 
Objects:

person: 100% 

FPS:35.3 	 AVG_FPS:35.2

 cvWriteFrame 
Objects:

person: 100% 

FPS:35.3 	 AVG_FPS:35.2

 cvWriteFrame 
Objects:

person: 100% 

FPS:35.1 	 AVG_FPS:35.2

 cvWriteFrame 
Objects:

person: 100% 

FPS:35.2 	 AVG_FPS:35.2

 cvWriteFrame 
Objects:

person: 100% 

FPS:35.0 	 AVG_FPS:35.2

 cvWriteFrame 
Objects:

person: 100% 

FPS:34.6 	 AVG_FPS:35.2

 cvWriteFrame 
Objects:

person: 100% 

FPS:34.6 	 AVG_FPS:35.2

 cvWriteFrame 
Objects:

person: 100% 

FPS:34.7 	 AVG_FPS:35.2

In [15]:
from google.colab import files
files.download('output1.avi')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>